In [1]:
import numpy as np
import tensorflow as tf

def sigmoid(x):
    return 1/(1+np.exp(-x))

In [2]:
data_x1 = np.array([[0., 1.], [1, 0], [1, 1], [0, -1], [-1, 0], [-1, -1], [-1,-1]])
data_x2 = np.array([[1., 0.], [-1, 1], [0, -2], [2, 1], [1, -1], [2, 2], [1, 1]])
data_xx = np.array([data_x1, data_x2])

n_inputs = data_x1.shape[0]
n_nodes = 3
n_steps = 2

state_0 = np.zeros([n_inputs, n_nodes])
state_0

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

sigmoind(Wx + Uy + b)

In [3]:
W = np.array([[1, 0, -1], [2, 2, 1]])
U = np.array([[1, 1, 0], [0, -2, -1], [-1, 0, 1]])
b = np.array([0.5, -0.5, 1])

WU = np.concatenate([W, U])
WU

array([[ 1,  0, -1],
       [ 2,  2,  1],
       [ 1,  1,  0],
       [ 0, -2, -1],
       [-1,  0,  1]])

In [4]:
output_weights = np.array([[2], [-0.5], [3]])
output_bias = np.array([[-3]])

In [5]:
def getState(data_x, state, WU, b):
    data = np.concatenate([data_x, state], axis=1)
    return sigmoid(np.matmul(data, WU) + b)

def getOutput(state, output_weights, output_bias):
    return sigmoid(np.matmul(state, output_weights) + output_bias)

state_1 = getState(data_x1, state_0, WU, b)
state_2 = getState(data_x2, state_1, WU, b)
output_1 = getOutput(state_1, output_weights, output_bias)
output_2 = getOutput(state_2, output_weights, output_bias)

print(state_1)
print(state_2)
print(output_1)
print(output_2)

[[ 0.92414182  0.81757448  0.88079708]
 [ 0.81757448  0.37754067  0.5       ]
 [ 0.97068777  0.81757448  0.73105858]
 [ 0.18242552  0.07585818  0.5       ]
 [ 0.37754067  0.37754067  0.88079708]
 [ 0.07585818  0.07585818  0.73105858]
 [ 0.07585818  0.07585818  0.73105858]]
[[ 0.82395043  0.2295229   0.51580039]
 [ 0.86027483  0.82671081  0.95780972]
 [ 0.03695601  0.00568365  0.25227493]
 [ 0.98496796  0.82211005  0.60447392]
 [ 0.26830165  0.05327504  0.37830624]
 [ 0.99711344  0.96845057  0.83959266]
 [ 0.94504924  0.8059869   0.83959266]]
[[ 0.74685103]
 [ 0.48659787]
 [ 0.67387503]
 [ 0.23629906]
 [ 0.551987  ]
 [ 0.33335782]
 [ 0.33335782]]
[[ 0.52012426]
 [ 0.76505988]
 [ 0.10228358]
 [ 0.59201527]
 [ 0.20503531]
 [ 0.73667922]
 [ 0.73221368]]


In [6]:
inputs = tf.placeholder(tf.float32, [n_steps, n_inputs, 2])
inputs_seq = tf.unstack(inputs)

with tf.name_scope('rec_layer'):
    basic_rnn_cell = tf.contrib.rnn.core_rnn_cell.BasicRNNCell(num_units=3, activation=tf.sigmoid)
    basic_rnn_output, state = tf.contrib.rnn.static_rnn(
        cell=basic_rnn_cell, inputs=inputs_seq, dtype=tf.float32)

## Assign rnn weights and bias
cell_weight = [v for v in tf.trainable_variables() if v.name == 'rnn/basic_rnn_cell/weights:0'][0]
update_weight = tf.assign(cell_weight, WU)
cell_bias = [v for v in tf.trainable_variables() if v.name == 'rnn/basic_rnn_cell/biases:0'][0]
update_bias = tf.assign(cell_bias, b)  

In [7]:
with tf.name_scope('hidden_layer'):
    tf_output_weights = tf.constant(output_weights, dtype=tf.float32)
    tf_output_bias = tf.constant(output_bias, dtype=tf.float32)
    rnn_outs = tf.concat(tf.unstack(basic_rnn_output), axis=0)
    output = tf.reshape(tf.sigmoid(tf.matmul(rnn_outs, tf_output_weights) + tf_output_bias), [2, n_inputs, -1])

In [8]:
feed_dict = {inputs: data_xx}

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print('assigned values')
    sess.run(update_weight)
    sess.run(update_bias)
    outs = sess.run(rnn_outs, feed_dict)
    print(outs)
    print(sess.run(output, feed_dict))

assigned values
[[ 0.92414182  0.81757444  0.88079703]
 [ 0.81757444  0.37754068  0.5       ]
 [ 0.97068775  0.81757444  0.7310586 ]
 [ 0.18242553  0.07585818  0.5       ]
 [ 0.37754068  0.37754068  0.88079703]
 [ 0.07585818  0.07585818  0.7310586 ]
 [ 0.07585818  0.07585818  0.7310586 ]
 [ 0.82395041  0.22952288  0.51580036]
 [ 0.86027485  0.82671082  0.95780975]
 [ 0.03695601  0.00568365  0.25227493]
 [ 0.98496801  0.82211006  0.60447395]
 [ 0.26830167  0.05327505  0.37830624]
 [ 0.99711347  0.96845055  0.8395927 ]
 [ 0.94504929  0.80598688  0.8395927 ]]
[[[ 0.74685109]
  [ 0.4865979 ]
  [ 0.67387503]
  [ 0.23629905]
  [ 0.55198699]
  [ 0.33335781]
  [ 0.33335781]]

 [[ 0.5201242 ]
  [ 0.76505989]
  [ 0.10228358]
  [ 0.59201539]
  [ 0.20503531]
  [ 0.73667932]
  [ 0.73221374]]]


In [9]:
print(outs)

[[ 0.92414182  0.81757444  0.88079703]
 [ 0.81757444  0.37754068  0.5       ]
 [ 0.97068775  0.81757444  0.7310586 ]
 [ 0.18242553  0.07585818  0.5       ]
 [ 0.37754068  0.37754068  0.88079703]
 [ 0.07585818  0.07585818  0.7310586 ]
 [ 0.07585818  0.07585818  0.7310586 ]
 [ 0.82395041  0.22952288  0.51580036]
 [ 0.86027485  0.82671082  0.95780975]
 [ 0.03695601  0.00568365  0.25227493]
 [ 0.98496801  0.82211006  0.60447395]
 [ 0.26830167  0.05327505  0.37830624]
 [ 0.99711347  0.96845055  0.8395927 ]
 [ 0.94504929  0.80598688  0.8395927 ]]


In [11]:
## Prinitng variables

feed_dict = {inputs: data_xx}

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    variables_names =[v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    print('assigned values')
    for k,v in zip(variables_names, values):
        print(k, v)
    sess.run(update_weight)
    sess.run(update_bias)
    print('updated')
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    print('assigned values')
    for k,v in zip(variables_names, values):
        print(k, v)
    out, st = sess.run((output, state), feed_dict)
    print('output1')
    print(out[0])
    print('output2')
    print(out[1])
    print('state')
    print(st)

assigned values
rnn/basic_rnn_cell/weights:0 [[ 0.40049845  0.31158358 -0.55095088]
 [-0.32545346  0.48703271  0.33160609]
 [-0.23697025  0.83945352 -0.70235574]
 [-0.59139669 -0.3273896   0.00201374]
 [-0.60951608  0.53296405 -0.80411094]]
rnn/basic_rnn_cell/biases:0 [ 0.  0.  0.]
updated
assigned values
rnn/basic_rnn_cell/weights:0 [[ 1.  0. -1.]
 [ 2.  2.  1.]
 [ 1.  1.  0.]
 [ 0. -2. -1.]
 [-1.  0.  1.]]
rnn/basic_rnn_cell/biases:0 [ 0.5 -0.5  1. ]
output1
[[ 0.74685109]
 [ 0.4865979 ]
 [ 0.67387503]
 [ 0.23629905]
 [ 0.55198699]
 [ 0.33335781]
 [ 0.33335781]]
output2
[[ 0.5201242 ]
 [ 0.76505989]
 [ 0.10228358]
 [ 0.59201539]
 [ 0.20503531]
 [ 0.73667932]
 [ 0.73221374]]
state
[[ 0.82395041  0.22952288  0.51580036]
 [ 0.86027485  0.82671082  0.95780975]
 [ 0.03695601  0.00568365  0.25227493]
 [ 0.98496801  0.82211006  0.60447395]
 [ 0.26830167  0.05327505  0.37830624]
 [ 0.99711347  0.96845055  0.8395927 ]
 [ 0.94504929  0.80598688  0.8395927 ]]
